# CGV Review Crawling 

1-1. 관련 라이브러리 설치

In [6]:
#pip install selenium
# 크롬브라우저 업데이트 될 때마다 driver 재설치를 하지 않기 위해 설치
#pip install webdriver-manager  

1-2. library import 

In [7]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

import warnings # 경고창 무시
warnings.filterwarnings('ignore')

1-3. crawling: extract 

In [8]:
def get_movie_reviews(url, page_num=12):
    chrome_driver =  'C:/Users/user-pc/Downloads/chromedriver_win32/chromedriver.exe'
    wd = webdriver.Chrome(chrome_driver)

    wd.get(url)
    # 빈 리스트 생성하기
    writer_list=[]
    review_list=[]
    date_list=[]
    like_list=[]
    
    for page_no in range(1,page_num+1): # 1페이지에서 page_num까지의 리뷰 추출
        try:
            page_ul = wd.find_element_by_id('paging_point') # 페이지 포인트 코드 추출
            page_a = page_ul.find_element_by_link_text(str(page_no))
            page_a.click()
            time.sleep(2) # 페이지 로딩까지의 시간 두기

            writers = wd.find_elements_by_class_name('writer-name')
            writer_list += [writer.text for writer in writers]
            
            reviews = wd.find_elements_by_class_name('box-comment')
            review_list += [ review.text for review in reviews ]
            
            dates = wd.find_elements_by_class_name('day')
            date_list += [date.text for date in dates]
            
            likes = wd.find_elements_by_id('idLikeValue')
            like_list += [like.text for like in likes]            
                    
            if page_no%10==0: # 10이상의 값을 만났을 때 다음 페이지로 넘기기 버튼
                    if page_no==10:
                        next_button = page_ul.find_element_by_class_name("paging-side")
                        #next_button = page_ul.find_element_by_class_name('btn-paging next')
                        next_button.click()
                        time.sleep(2)
                    else:
                        next_button = page_ul.find_element_by_xpath('//*[@id="paging_point"]/li[13]/button').click()
                        time.sleep(2)
        except NoSuchElementException:
                break
    movie_review_df = pd.DataFrame({"Writer" : writer_list,
                                   "Review" : review_list,
                                   "Date" : date_list,
                                   "Like" : like_list})
    wd.close()
    return movie_review_df


1-5. function request

In [9]:
# url 과 pagenum 을 입력하여 작성자, 리뷰내용, 작성날짜, 좋아요 수 요청 및 dataframe 반환
#범죄도시4 movie_review_df = get_movie_reviews("http://www.cgv.co.kr/movies/detail-view/?midx=88104#", page_num=3905)
movie_review_df = get_movie_reviews("http://www.cgv.co.kr/movies/detail-view/?midx=85813", page_num=12)

In [10]:
movie_review_df

,Writer,Review,Date,Like
0,sa**136,1편과 비슷하게 악날하고 다씹어먹내요.,2024.07.14,0
1,garylove,너무 재밌었어요 역시 범죄도시,2024.07.12,0
2,al**n1848,마요미와 손석구는 진리다,2024.07.09,0
3,Yㅖ지,세번째같이본영화입니당,2024.07.08,0
4,pa**o1215,기대이상으로 너무 좋았어요,2024.07.08,1
...,...,...,...,...
67,jr**5673,마동석 핵펀치 보러 가는 거임 대체 불가함,2024.05.06,0
68,유자차두잔,무섭지만 액션은 잼나고 계속 시리즈가즈아,2024.05.06,0
69,참치코치,역시나 재밌었음 시리즈물👍,2024.05.06,0
70,ka**i1008,손석구 연기 짱! 최귀화 마동석 케미 좋음,2024.05.04,0


In [16]:
movie_review_df.to_csv('범죄도시2크롤링.csv', index=False, encoding="utf-8-sig")